# dataset class  transforms

In [0]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
path = "/content/drive/My Drive/data/"
from os import listdir
from os.path import isfile, join
import h5py
import numpy as np
partition = [f for f in listdir(path) if isfile(join(path, f))]
SequenceDB, LabelsDB =[], []
for i, sequence in enumerate(partition):
  if i % 50 == 0:
    print(i)
  with h5py.File(path + '/' + sequence, 'r') as f:
    SequenceDB.append(np.stack(tuple(f['ch{}'.format(i)] for i in range(4)), axis = -1 ))
    LabelsDB.append(f['label'][()])


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350


In [0]:
from __future__ import print_function, division
import os, sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import json
from random import shuffle
class Data_Sequences(Dataset):
    def __init__(self, root_dir, SequenceDB, LabelsDB, transform=None):
        """
        Args:
            root_dir (string): Directory with all the h5 sequences.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.SequenceDB, self.LabelsDB = SequenceDB, LabelsDB
        self.transform = transform
        
        
    def __len__(self):
        return len(self.SequenceDB)
    def __getitem__(self, index):
        data = self.SequenceDB[index]
        label = self.LabelsDB[index]
        """with h5py.File(self.root_dir + '/' + sequence, 'r') as f:
            data = np.stack(tuple(f['ch{}'.format(i)] for i in range(4)), axis = -1 )
            label = f['label'][()]"""
        if self.transform != None:
            data, label = self.transform((data, label))# make the necessary transformations
        return data, label
    
    
    
            
class Reshape(object):
    '''
    reshape an image to 32 * 32 
    '''
    def __call__(self, sample):
        data, label = sample
        # a whole sequence 
        data = data.reshape((data.shape[0], 32,32, 4))
        return data, label
class Rescale(object):
    
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple): Desired output size. If tuple, output is
            matched to output_size. 
    """

    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        newH, newW = self.output_size
        data, label = sample
        data = data.reshape((data.shape[0], 32,32, 4))
        #data = transform.resize(data, (data.shape[0], newH, newW, data.shape[3]))
        return data, label

class ToTensor(object):
    """transposes ndarrays in sample to Tensors.
        and transforms them to tensors
    """

    def __call__(self, sample):
        data, label= sample
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        data = data.transpose((0, 2, 1))
        return torch.from_numpy(data),torch.from_numpy(label)

class SameSize(object):
    " either completes a sequence with pads or subsamples it"
    def __init__(self, size):
        self.size = size
    def __call__(self, sample):
        data, labels = sample
        sequences = np.zeros((self.size, *data.shape[1:]))
        labels_2 = np.zeros((self.size, *labels.shape[1:]))
        sequences[-data.shape[0]:] = data[:self.size]
        labels_2[-labels.shape[0]:] = labels[:self.size]
        return sequences, labels_2
        



In [0]:
params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 8}
t = transforms.Compose([SameSize(40), ToTensor()]) # composition of transformations
training_set = Data_Sequences(path,SequenceDB,LabelsDB, transform = t) # open the dataset
training_generator = DataLoader(training_set, **params)

training_set = Data_Sequences(path,SequenceDB,LabelsDB, transform = t) # open the dataset
training_generator = DataLoader(training_set, **params)



# definition of model

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.nn.functional import softmax

class LSTM(nn.Module):
    def __init__(self, batch_size, seq_len, input_dim, n_layers, hidden_dim, lstm_dropout, Dropout):
        super(LSTM, self).__init__()
        self.batch_size, self.seq_len, self.input_dim, self.n_layers, self.hidden_dim, self.lstm_dropout, self.Dropout = batch_size, seq_len, input_dim, n_layers, hidden_dim, lstm_dropout, Dropout
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, 1, dropout=lstm_dropout, batch_first=True)
        self.dropout = nn.Dropout(Dropout)
        self.fc2 = nn.Linear(hidden_dim, 11)
        
    def forward(self, x):
        a = self.fc1(x.view(-1, seq_len, input_dim).float())
        out, hidden = self.lstm(a)
        out = self.fc2(out)
        out = self.dropout(out)
        out = softmax(out, dim = 2)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

# Train

In [0]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [0]:
batch_size =  16
seq_len = 40
input_dim = 4*32*32
n_layers = 1
hidden_dim = 512
model = LSTM(batch_size, seq_len, input_dim, n_layers, hidden_dim, 0, 0.5).to(device)

In [0]:

lr=0.005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 1
counter = 0
print_every = 100
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in training_generator:
        counter += 1
        print(counter)
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device=device, dtype=torch.int64)
        model.zero_grad()
        output, h = model(inputs)
        print(output.view(-1, 11).shape, labels.view(-1, 1).shape)
        loss = criterion(output.view(-1, 11), labels.view(-1))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            #val_h = model.init_hidden(batch_size) 
            #val_losses = []
            #model.eval()
            """for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)"""

1
torch.Size([640, 11]) torch.Size([640, 1])
2
torch.Size([640, 11]) torch.Size([640, 1])
3
torch.Size([640, 11]) torch.Size([640, 1])
4
torch.Size([640, 11]) torch.Size([640, 1])
5
torch.Size([640, 11]) torch.Size([640, 1])
6
torch.Size([640, 11]) torch.Size([640, 1])
7
torch.Size([640, 11]) torch.Size([640, 1])
8
torch.Size([640, 11]) torch.Size([640, 1])
9
torch.Size([640, 11]) torch.Size([640, 1])
10
torch.Size([640, 11]) torch.Size([640, 1])
11
torch.Size([640, 11]) torch.Size([640, 1])
12
torch.Size([640, 11]) torch.Size([640, 1])
13
torch.Size([640, 11]) torch.Size([640, 1])
14
torch.Size([640, 11]) torch.Size([640, 1])
15
torch.Size([640, 11]) torch.Size([640, 1])
16
torch.Size([640, 11]) torch.Size([640, 1])
17
torch.Size([640, 11]) torch.Size([640, 1])
18
torch.Size([640, 11]) torch.Size([640, 1])
19
torch.Size([640, 11]) torch.Size([640, 1])
20
torch.Size([640, 11]) torch.Size([640, 1])
21
torch.Size([640, 11]) torch.Size([640, 1])
22
torch.Size([640, 11]) torch.Size([640, 1